In [3]:
import sqlite3
import openai
!pip install langchain_community
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Set up OpenAI API (Ensure your API key is set properly)
openai.api_key = #"your API key"  # Remove before submission

# Database setup
conn = sqlite3.connect("language_learning.db")
cursor = conn.cursor()

# Create a table to store mistakes
cursor.execute("""
CREATE TABLE IF NOT EXISTS mistakes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_input TEXT,
    correct_suggestion TEXT,
    learning_language TEXT
)
""")
conn.commit()

# Initialize the chatbot
chatbot = ChatOpenAI(model_name="gpt-4", openai_api_key="your API key")

# Database setup
conn = sqlite3.connect("language_learning.db")
cursor = conn.cursor()

def get_user_preferences():
    """Ask the user for language learning preferences."""
    learning_language = input("Which language do you want to learn? ")
    known_language = input("Which language do you already know? ")
    level = input("What is your current level in this language? (Beginner, Intermediate, Advanced) ")

    return learning_language, known_language, level

def correct_mistake(user_input, learning_language):
    """Use GPT to correct user input."""
    prompt = f"You are a language tutor. Correct this sentence in {learning_language} and explain the mistake:\n\n{user_input}"
    response = chatbot([HumanMessage(content=prompt)])

    return response.content

def save_mistake(user_input, correction, learning_language):
    """Save mistakes to SQLite database."""
    cursor.execute("INSERT INTO mistakes (user_input, correct_suggestion, learning_language) VALUES (?, ?, ?)",
                   (user_input, correction, learning_language))
    conn.commit()

def review_mistakes(learning_language):
    """Provide a summary of mistakes made."""
    cursor.execute("SELECT user_input, correct_suggestion FROM mistakes WHERE learning_language=?", (learning_language,))
    mistakes = cursor.fetchall()

    if not mistakes:
        print("Great job! No mistakes recorded.")
        return

    print("\nHere are the mistakes you made:")
    for idx, (mistake, correction) in enumerate(mistakes, 1):
        print(f"{idx}. Incorrect: {mistake}\n   Corrected: {correction}\n")

def chatbot_session(learning_language):
    """Main chat session with correction feature."""
    print(f"\nStarting conversation in {learning_language}. Type 'exit' to end.")

    while True:
        user_input = input(f"You ({learning_language}): ")
        if user_input.lower() == 'exit':
            break

        correction = correct_mistake(user_input, learning_language)
        print(f"Chatbot: {correction}")

        save_mistake(user_input, correction, learning_language)

def main():
    """Run the chatbot system."""
    learning_language, known_language, level = get_user_preferences()

    print(f"\nWelcome! You will be chatting in {learning_language}. I'll help you with corrections.")
    chatbot_session(learning_language)

    review_mistakes(learning_language)
    conn.close()

if __name__ == "__main__":
    main()


Which language do you want to learn? panjabi
Which language do you already know? hindi
What is your current level in this language? (Beginner, Intermediate, Advanced) advanced

Welcome! You will be chatting in panjabi. I'll help you with corrections.

Starting conversation in panjabi. Type 'exit' to end.
You (panjabi): exit
Great job! No mistakes recorded.
